# Tales RAG and PowerPoint Agent Evaluation

This notebook demonstrates how to evaluate the RAG agent and PowerPoint generation capabilities of the Tales system.

In [1]:
from tales.evaluation import RAGEvaluator
from tales.db_handler import ChromaDBHandler
from tales.config import DB_PATH

# Initialize the RAG evaluator
rag_evaluator = RAGEvaluator()

Compiling RAG Agent...
Database Initialized Successfully...


## Check Available Documents

First, let's check what documents are available in our vector store.

In [2]:
# Initialize the ChromaDB handler
db_handler = ChromaDBHandler(persist_directory=DB_PATH)

# Get stored documents
stored_docs = db_handler.get_stored_documents()
print(f"Found {len(stored_docs)} documents in the vector store:")
for doc in stored_docs:
    print(f" - {doc}")

Database Initialized Successfully...
Found 2 documents in the vector store:
 - data/How Much Information.pdf
 - data/bugra.pdf


## Single Query Evaluation

Let's evaluate the RAG agent on a single query.

In [3]:

# Define a query to evaluate
query = "Who is Bugra?"

# Run the evaluation
rag_metrics, messages = rag_evaluator.evaluate_rag_query(query)

# Print the metrics"
print("=== RAG Evaluation Results ===")
print(f"Response Time: {rag_metrics.response_time:.2f} seconds")
print(f"Context Relevance: {rag_metrics.context_relevance_score:.1f}/10")
print(f"Answer Correctness: {rag_metrics.answer_correctness_score:.1f}/10")
print(f"Answer Completeness: {rag_metrics.answer_completeness_score:.1f}/10")
print(f"Hallucination Score: {rag_metrics.hallucination_score:.1f}/10 (lower is better)")
print(f"Documents Retrieved: {rag_metrics.num_documents_retrieved}")

Analyzing query...
Retrieving documents...
Generating answer...
Reflecting on answer...
{'context': [Document(id='07b92f5e-fa22-4eba-843f-ec47a4d003bf', metadata={'page': 0, 'page_label': '1', 'source': 'data/bugra.pdf'}, page_content='Bugra is a male student in National college of Ireland, he is 25 years old, he likes \nelectronic music and is a addicted to drugs.'), Document(id='df3ca118-7f7c-44aa-976f-8a73a56539b5', metadata={'page': 0, 'page_label': '1', 'source': 'data/How Much Information.pdf'}, page_content='How Much Information?\nEffects of Transparency on Trust in an Algorithmic Interface\nRen´e F. Kizilcec\nDepartment of Communication, Stanford University\nkizilcec@stanford.edu\nABSTRACT\nThe rising prevalence of algorithmic interfaces, such as cu-\nrated feeds in online news, raises new questions for designers,\nscholars, and critics of media. This work focuses on how trans-\nparent design of algorithmic interfaces can promote awareness\nand foster trust. A two-stage process

## View the RAG Response

Let's look at the response the RAG agent provided.

In [4]:
# Print the last message (the response)
from IPython.display import Markdown

response = next((msg.content for msg in reversed(messages) if hasattr(msg, 'content')), "No response found")
display(Markdown(f"**Query:** {query}\n\n**Response:**\n{response}"))

**Query:** Who is Bugra?

**Response:**
Bugra is a 25-year-old male student at the National College of Ireland who enjoys electronic music and is addicted to drugs.